# Programming Assignment

This programming assignment is migrated from tensorflow 2.0 exercise, aiming at familiar with pytorch programming API

Reference:
* https://nextjournal.com/gkoehler/pytorch-mnist
* https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
* https://blog.csdn.net/touristourist/article/details/100535544
* Transform - https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Torch wrapper for similar APIs to keras - https://github.com/ncullen93/torchsample
* Torch weight and bias initialization - https://androidkt.com/initialize-weight-bias-pytorch/
* Torch regularization - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-l1-l2-and-elastic-net-regularization-with-pytorch.md

## Model validation on the Iris dataset

#### The Iris dataset

In this assignment, you will use the [Iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html). It consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. For a reference, see the following papers:

- R. A. Fisher. "The use of multiple measurements in taxonomic problems". Annals of Eugenics. 7 (2): 179–188, 1936.

Your goal is to construct a neural network that classifies each sample into the correct class, as well as applying validation and regularisation techniques.

In [ ]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn import datasets, model_selection 

#### Load and preprocess the data

In [ ]:
from typing import Optional, Callable, Dict
from sklearn.model_selection import train_test_split

class TensorDataSet(Dataset):
    def __init__(self, inputs, targets, 
                 transform: Optional[Callable]=None,
                 target_transform: Optional[Callable]=None):
        self.inputs = inputs
        self.targets = targets
        self.transform = transforms
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        input_data, input_target = self.inputs[idx], self.targets[idx]
        if self.transform is not None:
            input_data = self.transform(input_data)
        if self.target_transform is not None:
            input_target = self.target_transform(input_target)
        return input_data, input_target
    
class IrisData():
    """ Iris training, validation or testing dataset load from sklearn
    """
    default_batch_size:int=64
    
    def __init__(self, transforms: Dict[str, Optional[Callable]], target_transforms: Dict[str, Optional[Callable]], 
                 batch_size: Dict[str, int],
                 train_test_split: float=0.15, valid_train_split: float=0.2,):
        train_data, train_target, valid_data, valid_target, test_data, test_target = self._read_in_and_split_data(train_test_split, valid_train_split)
        self.train_dataset = TensorDataSet(train_data, train_target, 
                                           transforms.get("train", None), target_transforms.get("train", None))
        self.valid_dataset = TensorDataSet(valid_data, valid_target, 
                                           transforms.get("valid", None), target_transforms.get("valid", None))
        self.test_dataset = TensorDataSet(test_data, test_target, 
                                          transforms.get("test", None), target_transforms.get("test", None))
        self.train_loader = DataLoader(self.train_dataset, 
                                       batch_size=batch_size.get("train", default_batch_size), 
                                       shuffle=True)
        self.valid_loader = DataLoader(self.valid_dataset, 
                                       batch_size=batch_size.get("valid", default_batch_size), 
                                       shuffle=True)
        self.test_loader = DataLoader(self.test_dataset, 
                                      batch_size=batch_size.get("test", default_batch_size), 
                                      shuffle=True)
        
    def _read_in_and_split_data(train_test_split, valid_train_split):
        iris_data = datasets.load_iris()
        tranvalid_data, test_data, tranvalid_target, test_target = train_test_split(iris_data.data, iris_data.target,
                                                                                    test_size=train_test_split,
                                                                                    random_state=1)
        train_data, valid_data, train_target, valid_target = train_test_split(tranvalid_data, tranvalid_target,
                                                                              test_size=valid_train_split,
                                                                              random_state=1)
        return train_data, train_target, valid_data, valid_target, test_data, test_target
    
    def get_train(self):
        return self.train_loader, self.train_dataset
    
    def get_valid(self):
        return self.valid_loader, self.valid_dataset
    
    def get_test(self):
        return self.test_loader, self.test_dataset

In [ ]:
class RegularizedNetwork(nn.Module):
    def __init__(self, dropout_rate: float):
        super(RegularizedNetwork, self).__init__()
        self.first_linear_layer = nn.LazyLinear(64)
        self.regularized_stack = nn.Sequential(
            self.first_linear_layer, nn.ReLU(),
            nn.LazyLinear(128), nn.ReLU(),
            nn.LazyLinear(128), nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.LazyLinear(128), nn.ReLU(),
            nn.LazyLinear(128), nn.ReLU(),
            nn.LazyBatchNorm2d(),
            nn.LazyLinear(64), nn.ReLU(),
            nn.LazyLinear(64), nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.LazyLinear(64), nn.ReLU(),
            nn.LazyLinear(64), nn.ReLU(),
            nn.LazyLinear(3),
            nn.Softmax(dim=1),
        )
        # intialization for weight and bias
        nn.init.kaiming_uniform_(self.first_linear_layer.weight.data)
        nn.init.constant_(self.first_linear_layer.bias.data, 1)
    
    def forward(self, x):
        return self.regularized_stack(x)